Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы.

In [2]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

Алгоритм работает очень медленно, если словарь большой, поэтому уменьшим его объем.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True, min_df = 120)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=120,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'however', 'still', 'also', 'there', 'side', 'cannot', 'formerly', 'than', 'while', 'bill', 'besides', 'hundred', 're', 'thick', 'both', 'thin', 'those', 'three', 'whither', 'behind', 'same', 'any', 'mill', 'always', 'above', 'anything', 'often', 'our', 'me', 'latter', 'become'...e', 'thru', 'yourself', 'rather', 'noone', 'hereafter', 'together', 'beside', 'so', 'thence', 'ie'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Введем функцию, которая ранжирует элементы по их весу.

In [20]:
def range_weights(weights):
    norms = np.sort(weights) / np.sum(weights)
    bounds = np.cumsum(norms)

    rand = np.random.rand()
    for i in range(len(weights)):
        if(rand < bounds[i]):
            rand = np.argsort(weights)[i]
            break;
    return rand

In [21]:
tagofword = np.zeros(len(vectorizer.vocabulary_), dtype = int) 
voltag = np.zeros(len(newsgroups_train.target_names))           
numwordintag = np.zeros((len(newsgroups_train.target_names), len(vectorizer.vocabulary_)))                                                           
numwordsintagtxt = np.zeros((len(newsgroups_train.data), len(newsgroups_train.target_names)))                                          

alpha = np.zeros(len(newsgroups_train.target_names))         
beta = np.zeros((len(newsgroups_train.target_names), len(vectorizer.vocabulary_)))  
                                                                

# случайно распределим слова по тэгам
for i in range(len(vectorizer.vocabulary_)):       
    tagofword[i] = range_weights(np.full(20, 1/20))
    
for i in range(len(newsgroups_train.data)):
    alpha[newsgroups_train.target[i]] = alpha[newsgroups_train.target[i]] + 1
    doc = newsgroups_train.data[i]
    beta[newsgroups_train.target[i]] = beta[newsgroups_train.target[i]] + vectorizer.transform([doc])
    
    x = np.resize(vectorizer.transform([doc]).toarray(), len(vectorizer.vocabulary_))
    b = np.argwhere(x)
    c = tagofword[b]
    for j in range(len(voltag)):
        numwordsintagtxt[i, j] = len(c[(c == j)])
        voltag[j] = voltag[j] + len(c[(c == j)])
    doc_transformed = vectorizer.inverse_transform(vectorizer.transform([doc]))[0]
    for j in range(len(doc_transformed)):
        word = vectorizer.vocabulary_.get(doc_transformed[j])
        numwordintag[tagofword[word], word] = numwordintag[tagofword[word], word] + 1
        

In [22]:
for count in range(50):                                         
    for i in range(len(newsgroups_train.data)):
        doc = newsgroups_train.data[i]
        doc_transformed = vectorizer.inverse_transform(vectorizer.transform([doc]))[0]
        for j in range(len(doc_transformed)):
            word = vectorizer.vocabulary_.get(doc_transformed[j])
            tag = tagofword[word]
            numwordsintagtxt[i, tag] = numwordsintagtxt[i, tag] - 1
            voltag[tag] = voltag[tag] - 1
            numwordintag[tag, word] = numwordintag[tag, word] - 1
            #
            p = np.zeros(len(voltag))
            for k in range(len(voltag)):
                p[k] = (numwordsintagtxt[i, k] + alpha[k]) * (numwordintag[k, word] + beta[k, word]) / (voltag[k] + np.sum(beta[k]))
            tag = range_weights(np.abs(p))
            tagofword[word] = tag
            numwordsintagtxt[i, tag] = numwordsintagtxt[i, tag] + 1
            voltag[tag] = voltag[tag] + 1
            numwordintag[tag, word] = numwordintag[tag, word] + 1

In [34]:
# наиболее часто встречающиеся слова по конкретному тэгу

InvDict = {v:k  for k,v in vectorizer.vocabulary_.items()}

for i in range(len(newsgroups_train.target_names)):
    print('Topic = {0}\n'.format(newsgroups_train.target_names[i]))
    x = np.argsort(beta[i]) [tagofword[np.argsort(beta[i])] == i] [:-10:-1]
    for j in range(len(x)):
        print(InvDict.get(x[j]), end = ' ')
    print()
    print()
    print('--------------------------------------------------------------------\n')

Topic = alt.atheism

know good doesn isn world ll statement stay considered 

--------------------------------------------------------------------

Topic = comp.graphics

file color called following hardware pub high sun technical 

--------------------------------------------------------------------

Topic = comp.os.ms-windows.misc

pc ftp try group 24 latest email drive mb 

--------------------------------------------------------------------

Topic = comp.sys.ibm.pc.hardware

thanks computer new board 486 hi memory data machine 

--------------------------------------------------------------------

Topic = comp.sys.mac.hardware

apple use problem video disk bit info modem chip 

--------------------------------------------------------------------

Topic = comp.windows.x

window server just display files function doing change request 

--------------------------------------------------------------------

Topic = misc.forsale

shipping 30 100 included list 16 selling windows 300 

---

-------------------------------------------------------------------------------------

Как мы видим, алгоритм производит сортировку слов по тэгам довольно действенно, даже с учетом того, что было произведено мало итераций (из-за большого объема данных алгоритм работает долго, поэтому пришлось сократить их количество, чтобы получить результат быстрее). Таким образом, чтобы получить качественные результаты работы программы, нужно увеличить количество итераций в несколько раз.